In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import datetime
from utils2 import seasonal_plotter
import os

# ******************************************************************************************
# ************************************   Part 1   ******************************************
# ******************************************************************************************

print(os.getcwd())

/root/forecasting/exploration/py


In [2]:

temps = pd.read_csv('../../data/vic_elec.csv', parse_dates=[['Date', 'Time']])
elec_demand = (
    temps
    .set_index('Date_Time')
    .drop(['Index', 'Date.1', 'Temperature', 'Holiday'], axis=1)
    .resample('1H')
    .sum()
)
elec_demand.head()

,Demand
Date_Time,
2012-01-01 00:00:00,8646.190700
2012-01-01 01:00:00,7926.529376
2012-01-01 02:00:00,7901.826990
2012-01-01 03:00:00,7255.721350
2012-01-01 04:00:00,6792.503352
